### colab으로 할 때만 실행

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!nvidia-smi

In [ ]:
import os
os.chdir("/content/drive/MyDrive")

### 설치 필요할때만

In [ ]:
!python --version
!pip install opencv-python
!pip install tensorflow_datasets
!pip install -U scikit-learn
!pip install adabelief-tf

### 기본 설정

In [ ]:
from shutil import copy
from collections import defaultdict
import scipy
import numpy as np
import matplotlib.pyplot as plt
import warnings

import random
import cv2
import glob
from PIL import Image, ImageEnhance
import PIL.ImageOps

import torch
import torchvision
from torch import nn


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.ops.clip_ops import clip_by_global_norm
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import tensorflow_datasets as tfds


# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization, Dropout, Flatten, Dense, concatenate
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
#from keras.optimizers import SGD

from keras_preprocessing.image import array_to_img, img_to_array, load_img

from sklearn.model_selection import train_test_split


from deeplab2.model.pixel_encoder import moat
from adabelief_tf import AdaBeliefOptimizer

### 데이터 다운로드 + split (한번만)

https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz

In [ ]:
#!tar -xvf "/Users/dayeonjung/Desktop/deep/main/tinyMOAT_Ensemble/cifar10.tar"
import splitfolders
splitfolders.ratio("cifar10/train/", output = "cifar10/output1/", seed = 1337, ratio = (0.84, 0.16))
splitfolders.ratio("cifar10/train/", output = "cifar10/output2/", seed = 1337, ratio = (0.42, 0.42, 0.16))

### 압축풀기

In [ ]:
!unzip -qq "/content/drive/MyDrive/split.zip"

### 초기 설정

In [ ]:
# 초기 변수 설정
n_classes = 10

### 이미지 크기 설정
img_width, img_height = 256,256

my_dir = os.getcwd()
train_data_dir = my_dir + '/split/train'
train1_data_dir = my_dir + '/split/train1'
train2_data_dir = my_dir + '/split/train2'
validation_data_dir = my_dir + '/split/val'
test_data_dir = my_dir + '/split/test'

nb_train_samples =  42000
nb_train1_samples =  21000
nb_train2_samples =  21000
nb_validation_samples = 8000
nb_test_samples = 10000
batch_size = 100

In [ ]:
# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

train1_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')   

# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)
train2_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

val_datagen = ImageDataGenerator(
    rescale=1./255,)

test_datagen = ImageDataGenerator(
    rescale=1./255,)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


train1_generator = train1_datagen.flow_from_directory(
    train1_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


train2_generator = train2_datagen.flow_from_directory(
    train2_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')
    

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='sparse')


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode=None)

Found 42000 images belonging to 10 classes.
Found 21000 images belonging to 10 classes.
Found 21000 images belonging to 10 classes.
Found 8000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


### 모델 생성

In [ ]:
moat1 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

o1 = moat1.output["stage5"]
r1 = tf.keras.layers.Flatten(name='flatten_layer1')(o1)
r1 = tf.keras.layers.BatchNormalization()(r1)
out1 = tf.keras.layers.Dense(
    units=10, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r1)  

def create_model1():
    return tf.keras.models.Model(moat1.input, out1)

In [ ]:
moat2 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

o2 = moat2.output["stage5"]
r2 = tf.keras.layers.Flatten(name='flatten_layer1')(o2)
r2 = tf.keras.layers.BatchNormalization()(r2)
out2 = tf.keras.layers.Dense(
    units=10, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r2)  

def create_model2():
    return tf.keras.models.Model(moat2.input, out2)

### Train 1

In [ ]:
warnings.filterwarnings('ignore')
model1 = create_model1()
#model1.load_weights('cifar10_final_disjoint_local_1_1.hdf5')


opt = AdaBeliefOptimizer(learning_rate=1e-3, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model1.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar10_final_disjoint_local_1_1.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar10_final_disjoint_local_1_1.log')

history = model1.fit(train1_generator,
                steps_per_epoch = nb_train1_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=10,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Epoch 1/10


ResourceExhaustedError: ignored

In [ ]:
model1 = create_model1()
model1.load_weights('cifar10_final_disjoint_local_1_1.hdf5')


opt = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model1.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar10_final_disjoint_local_1_2.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar10_final_disjoint_local_1_2.log')

history = model1.fit(train1_generator,
                steps_per_epoch = nb_train1_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=300,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])

In [ ]:
model1.save_weights('cifar10_final_disjoint_zoomx_1_2_final.hdf5')

### Train 2

In [ ]:
batch_size = 200
train2_generator = train2_datagen.flow(x=tr2_images, y=tr2_labels, batch_size = batch_size, shuffle=True)
validation_generator = val_datagen.flow(x=val_images, y=val_labels, batch_size = batch_size, shuffle=False)
warnings.filterwarnings('ignore')

model2 = create_model2()
#model2.load_weights('cifar10_final_disjoint_zoomx_2_1.hdf5')


opt = AdaBeliefOptimizer(learning_rate=1e-3, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model2.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar10_final_disjoint_zoomx_2_1.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar10_final_disjoint_zoomx_2_1.log')

history = model2.fit(train2_generator,
                steps_per_epoch = nb_train2_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=20,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])

In [ ]:
batch_size = 200
train2_generator = train2_datagen.flow(x=tr2_images, y=tr2_labels, batch_size = batch_size, shuffle=True)
validation_generator = val_datagen.flow(x=val_images, y=val_labels, batch_size = batch_size, shuffle=False)
warnings.filterwarnings('ignore')

model2 = create_model2()
model2.load_weights('cifar10_final_disjoint_zoomx_2_1.hdf5')


opt = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model2.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar10_final_disjoint_zoomx_2_2.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar10_final_disjoint_zoomx_2_2.log')

history = model2.fit(train2_generator,
                steps_per_epoch = nb_train2_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=300,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])

In [ ]:
model2.save_weights('cifar10_final_disjoint_zoomx_2_2_final.hdf5')

In [ ]:
moat2.save_weights('moat_cifar10_final_disjoint_zoomx_2_2_final.hdf5')